<h1 style="text-align:center;width:100%;background-color:#242424;padding-left:10px;margin-bottom:0px; color:skyblue"><font size = 5 color=skyblue>Segmenting and Clustering Neighborhoods in <br /> Torento</font></h1>
<h3 style="text-align:left;width:100%;background-color:#ddd;padding-left:10px;;margin-top:0px;"> <font size=1 color=#444>Developed by <font size=1 color=green>Reza Sadeghi Jafari </font>- March 2021 </font></h3>

# Introduction

This code developed in regard of Coursera assignment on course "Applied Data Science Capstone". It's required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information. The neighborhood data for torento is not readily available on the internet.

"For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe like the one shown below:

3. To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository.(10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas  to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe."

In [174]:
import pandas as pd
import numpy as np
import folium as fl
import geopy
from geopy.geocoders import Nominatim 
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [247]:
# I prefer to use pandas to fetch data from grid in HTML file, it's so fast and easy.
# I change the url to read from history, it seems changed during this month
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802'
dfs = pd.read_html(url)
df=dfs[0]   # it's retrive the first dataframe from all data frames which red by the "read_html" methode. 

In [248]:
df  # load Dataframe for review

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [249]:
df.describe() # I describe the dataframe to find some information about Dataframe

,Postal Code,Borough,Neighbourhood
count,180,180,180
unique,180,12,100
top,M4V,Not assigned,Not assigned
freq,1,77,77


In [250]:
df.info() # also .info would be helpful to review the null values, datatype, ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code      180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


<h3>start cleansing the data</h3>
<hr />
<span style='color:skyblue'>as requested by assumpsion we should remove the 'Not assigned' Borough from our records,</span>

In [251]:

Clean_df =df[df['Borough']!='Not assigned']
Clean_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<div style='color:skyblue'> To concatinate Neighbourhood values on same Borough values First i group by the dataframe by Borough then i used transform methode and lambda function on Neighbourhood values.</div>
<hr >


In [252]:
Clean_df['Neighbourhood']= Clean_df.groupby(['Postal Code','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
Clean_df =Clean_df.drop_duplicates() # remove duplicate if exist
Clean_df.head()

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<div style='color:skyblue'>As request, we should replace the 'Not assigned' values on Neighbourhood columns with Borough value </div>

In [253]:
Clean_df['Neighbourhood'].replace('Not assigned',Clean_df['Borough'],inplace=True)
Clean_df.rename(columns={'Postal Code': 'PCode'},inplace=True)
Clean_df

,PCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<div style='color:green'><h4> The result of cleansing </h4></div>

In [254]:
Clean_df.shape

(103, 3)